#  Bài tâp về mạng tích chập

Trong bài này, chúng ta sẽ xây dựng một mạng tích chập sử dụng torch và thử train&test với tập MNIST nhé.


## Tổng quan một mạng CNN cơ bản

![CNN](http://personal.ie.cuhk.edu.hk/~ccloy/project_target_code/images/fig3.png)

## MNIST dataset

Trong bài tập này, chúng ta sẽ sử dựng tập MNIST rất nổi tiếng vể  các chữ số viết tay từ 0->9. Tập dataset này bao gồm 60000 ảnh cho training và 10000 ảnh cho testing. Các bức ảnh này đều đã được căn giữa và chỉnh với kích thước cố định là 28x28.

Trong phần tiền xử lý, chúng ta sẽ cần chuẩn hóa các giá trị pixel của mỗi ảnh về khoảng [0,1], kiểu dữ liệu sẽ là float32

<!-- ![MNIST Dataset](http://neuralnetworksanddeeplearning.com/images/mnist_100_digits.png) -->

Chi tiết tại: http://yann.lecun.com/exdb/mnist/

# Some configs

- Chúng ta sẽ setup một số hyper-parameters cũng như một số giá trị cần dùng theo hướng dẫn nhé
- Ở đây, mình muốn các bạn sử dụng Cuda, hãy vào runtime, rồi change the runtime type sang GPU nhé 

In [1]:
# Trước hết, chúng ta import một số thư viện cần thiết đã
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
import random

In [2]:
# Số classes trong tập MNIST
num_classes = 10

# Số epoch 
epochs = 5

# Các tham số cần thiết cho quá trình traning.
learning_rate = 0.01
batch_size = 32
display_step = 10

# Path lưu best model 
checkpoint = 'cnnmodel.pth' # có thể để dạng *.pth

# device chúng ta dùng cuda
device = 'cuda' if torch.cuda.is_available() else 'cpu'
assert device == 'cuda' 

# Dataloader


In [3]:
# Transform image 
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)) 
    ])

# load dataset từ torchvision.datasets
train_dataset = datasets.MNIST('../data', train=True, download=True,transform=transform)
test_dataset = datasets.MNIST('../data', train=False,transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=batch_size)

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


# Model

- Trong bài này, chúng ta sẽ định nghĩa một class Net, nó sẽ có cấu trúc như hình ở đầu notebook
- Bạn hãy chỉnh các tham số cho phù hợp nhé :) 

In [4]:
class Net(nn.Module):
  def __init__(self):

    super(Net,self).__init__()
    self.dropout = nn.Dropout()
    self.relu = nn.ReLU()
    self.conv1 = nn.Conv2d(1,16,3,1,padding=1)
    self.maxpool1 = nn.MaxPool2d(2)
    self.conv2 = nn.Conv2d(16,32,3,1,padding=1)
    self.maxpool2 = nn.MaxPool2d(2)
    self.conv3 = nn.Conv2d(32,64,3,1,padding=1)
    self.maxpool3 = nn.MaxPool2d(2)
    self.fc1 = nn.Linear(64*3*3,256)
    # self.fc2 = nn.Linear(256,256)
    self.fc3 = nn.Linear(256,num_classes)

  def forward(self,x):
    # x = F.dropout(x,p=0.5,training=self.training)
    x = self.maxpool1(self.relu(self.conv1(x)))
    x = self.maxpool2(self.relu(self.conv2(x)))
    x = self.maxpool3(self.relu(self.conv3(x)))
    # x = F.dropout(x,p=0.5,training=self.training)
    x = self.dropout(x)
    x = x.view(-1,64*3*3)
    x = self.relu(self.fc1(x))
    x = self.relu(self.fc3(x))
    
    
    # x = F.log_softmax(x, dim=1)
    return x

In [5]:
# call model, đừng quên set device nhé
model = Net()
model.to(device)

# load lại pretrained model (nếu có)
# try:
#   model = torch.load(checkpoint)
# except:
#   print("!!! Hãy train để có checkpoint file")

Net(
  (dropout): Dropout(p=0.5, inplace=False)
  (relu): ReLU()
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=576, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=10, bias=True)
)

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = learning_rate)
best_val_loss = 999

for epoch in range(1,epochs+1):
    # Quá trình training 
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad() # zero_grad()
        output = model(data)
        loss = criterion(output,target)
        loss.backward() # backward()
        optimizer.step() # update
        if batch_idx % display_step == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    # Quá trình testing 
    model.eval()
    test_loss = 0
    correct = 0
    # set no grad cho quá trình testing
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            output =  F.log_softmax(output,1) # log softmax dùng F, chú ý dim
            test_loss += loss.item()
            pred = torch.argmax(output,1,keepdim=True) # argmax để lấy predicted label, chú ý dim, và keepdim = True
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset) 
    if test_loss < best_val_loss:
      best_val_loss = test_loss
      torch.save(model, checkpoint) # lưu lại model 
      print("***********    TEST_ACC = {}%    ***********".format(correct))

Train Epoch: 1 [0/60000 (0%)]	Train Loss: 1.379329
Train Epoch: 1 [320/60000 (1%)]	Train Loss: 1.172928
Train Epoch: 1 [640/60000 (1%)]	Train Loss: 1.295390
Train Epoch: 1 [960/60000 (2%)]	Train Loss: 1.246260
Train Epoch: 1 [1280/60000 (2%)]	Train Loss: 1.209984
Train Epoch: 1 [1600/60000 (3%)]	Train Loss: 1.382413
Train Epoch: 1 [1920/60000 (3%)]	Train Loss: 1.447459
Train Epoch: 1 [2240/60000 (4%)]	Train Loss: 1.393790
Train Epoch: 1 [2560/60000 (4%)]	Train Loss: 1.208150
Train Epoch: 1 [2880/60000 (5%)]	Train Loss: 1.093698
Train Epoch: 1 [3200/60000 (5%)]	Train Loss: 1.380430
Train Epoch: 1 [3520/60000 (6%)]	Train Loss: 1.225662
Train Epoch: 1 [3840/60000 (6%)]	Train Loss: 1.448376
Train Epoch: 1 [4160/60000 (7%)]	Train Loss: 1.164158
Train Epoch: 1 [4480/60000 (7%)]	Train Loss: 1.035121
Train Epoch: 1 [4800/60000 (8%)]	Train Loss: 1.146658
Train Epoch: 1 [5120/60000 (9%)]	Train Loss: 1.011565
Train Epoch: 1 [5440/60000 (9%)]	Train Loss: 1.091201
Train Epoch: 1 [5760/60000 (10%)]	

In [7]:
# load lại model đã train
model= torch.load(checkpoint)
# Set eval phase nhé bạn
model.eval()

Net(
  (dropout): Dropout(p=0.5, inplace=False)
  (relu): ReLU()
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=576, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=10, bias=True)
)

In [ ]:
# thử predict 

def plot(data,model):
  data = torch.unsqueeze(data,0).to(device) # unsqueeze data, đồng thời set device
  output = model(data)
  output = F.log_softmax(output,1) # log softmax, chú ý dim
  pred = torch.argmax(output,1,keepdim=True) # argmax, chú ý keepdim 
  print("Predict Number : ", pred[0][0].detach().cpu().numpy()) 
  plt.imshow(data[0][0].detach().cpu().numpy(),cmap='gray') 
  plt.show()

In [ ]:
item = iter(test_loader)
data,target = item.next() # lấy một batch ra
test_idx = random.choice(range(len(data))) # lấy index của một phần tử của một batch
data = data[test_idx]
target = target[test_idx]
assert data.shape == (1,28,28)
plot(data,model)